In [1]:
import pandas as pd
import numpy as np

from python.parameters import *
from python.components import *
from python.utils import *

In [2]:
components_csv = 'data/components.csv'

In [3]:
components_params = csvtoparameters(components_csv)